In [1]:
!pip install /kaggle/input/mypackages/mypackages/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
!pip install /kaggle/input/mypackages/mypackages/ultralytics-8.0.124-py3-none-any.whl
!pip install /kaggle/input/mypackages/mypackages/ensemble_boxes-1.0.9-py3-none-any.whl

Processing /kaggle/input/mypackages/mypackages/python_gdcm-3.0.22-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl
Processing /kaggle/input/mypackages/mypackages/ultralytics-8.0.124-py3-none-any.whl
Processing /kaggle/input/mypackages/mypackages/ensemble_boxes-1.0.9-py3-none-any.whl


In [2]:
import os
import sys

from PIL import Image
import pandas as pd
from tqdm.auto import tqdm
import numpy as np
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut

In [3]:
from numba import cuda
import gc
import torch
import time
import cv2
import PIL.Image
import random
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
from torch.utils.data import DataLoader, Dataset
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import albumentations
import timm
from collections import OrderedDict
from warnings import filterwarnings
filterwarnings("ignore")
device = torch.device('cuda')
torch.no_grad()

from glob import glob
import shutil, os
import matplotlib.pyplot as plt
from sklearn.model_selection import GroupKFold
import seaborn as sns

In [4]:
# hyperparameters
from scipy import optimize

class args:
    batch_size = 12
    n_worker = 8

    image_size = 512
    arch_name = 'tf_efficientnetv2_l_in21k'

    model_image_path = [
        '/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold0_epoch9_ap0.8534853452608039.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold1_epoch6_ap0.8691063684343019.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold2_epoch4_ap0.8526443347470971.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold3_epoch9_ap0.8485065639200695.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold4_epoch9_ap0.8745943231121824.pth',
    ]

    model_study_path = [
        '/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold0_epoch5_ap0.5511132228812636.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold1_epoch9_ap0.5633224707416896.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold2_epoch9_ap0.5755307073313223.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold3_epoch10_ap0.565790786617794.pth',
        '/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold4_epoch4_ap0.5755384751101597.pth',
    ]

    class_weight = [1,1,1,1,1,]

In [5]:
# 设置随机种子，以便实验复现

def set_seed(seed = 42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
set_seed()

# 1. dcm to png

In [6]:
# from https://www.kaggle.com/code/xhlulu/siim-covid-19-convert-to-jpg-256px?scriptVersionId=63196459
def read_xray(path, voi_lut = True, fix_monochrome = True):
    dicom = pydicom.read_file(path)

    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
               
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
        
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
        
    return data

In [7]:
def resize(array, size, keep_ratio=False, resample=Image.LANCZOS):
    # Original from: https://www.kaggle.com/xhlulu/vinbigdata-process-and-resize-to-image
    im = Image.fromarray(array)
    
    if keep_ratio:
        im.thumbnail((size, size), resample)
    else:
        im = im.resize((size, size), resample)
    
    return im

In [8]:
image_id = []
dim0 = []
dim1 = []
splits = []
study_id = []

split = 'test'

save_dir = f'/kaggle/working/tmp/{split}/'

os.makedirs(save_dir, exist_ok=True)
    
for dirname, _, filenames in tqdm(os.walk(f'/kaggle/input/siim-covid19-detection/{split}')):
    for file in filenames:
        # set keep_ratio=True to have original aspect ratio
        xray = read_xray(os.path.join(dirname, file))
        im = resize(xray, size=512)  
        im.save(os.path.join(save_dir, file.replace('dcm', 'png')))

        image_id.append(file.replace('.dcm', ''))
        dim0.append(xray.shape[0])
        dim1.append(xray.shape[1])
        splits.append(split)
        study_id.append(dirname.split('/')[-2])

0it [00:00, ?it/s]

In [9]:
meta = pd.DataFrame.from_dict({'image_id': image_id, 'dim0': dim0, 'dim1': dim1, 'split': splits, 'study_id':study_id})
meta = meta[meta['split'] == 'test']
meta['image_id'] = meta['image_id'] + '_image'
meta['study_id'] = meta['study_id'] + '_study'
meta.columns = ['id', 'dim0', 'dim1', 'split', 'study_id']
meta

,id,dim0,dim1,split,study_id
0,a29c5a68b07b_image,2320,2828,test,2fb11712bc93_study
1,9850b5470fd6_image,2330,2382,test,19c66935e737_study
2,8d6dea06a032_image,2422,3344,test,2fc50bf199cd_study
3,dfc5c09a50bc_image,1140,1387,test,97c5d6eb413d_study
4,7230234e120a_image,2318,2383,test,593c3f815635_study
...,...,...,...,...,...
1258,a43200bd5ceb_image,2336,2836,test,b12d2e5c39b7_study
1259,37bf83df1b86_image,3480,4240,test,6484393291ea_study
1260,b30d2aef985f_image,3480,4240,test,6484393291ea_study
1261,a37a362df0ac_image,3480,4240,test,531aa20ff7c3_study


In [10]:
all_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
all_df['id_last_str'] = all_df["id"].str[-1]

study_len = all_df[all_df['id_last_str'] == 'y'].shape[0]

all_df

,id,PredictionString,id_last_str
0,00188a671292_study,negative 1 0 0 1 1,y
1,004bd59708be_study,negative 1 0 0 1 1,y
2,00508faccd39_study,negative 1 0 0 1 1,y
3,006486aa80b2_study,negative 1 0 0 1 1,y
4,00655178fdfc_study,negative 1 0 0 1 1,y
...,...,...,...
2472,46719b856de1_image,none 1 0 0 1 1,e
2473,31c07523a69a_image,none 1 0 0 1 1,e
2474,f77d7d1aebab_image,none 1 0 0 1 1,e
2475,ccc5b63ca96d_image,none 1 0 0 1 1,e


# 2. 2-class Predict on Image Level

In [11]:
image_sub_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
image_sub_df = image_sub_df[study_len:]

image_sub_df["file_name"] = image_sub_df["id"].str.replace('_image', '') + ".png"
label_cols = '2_class_prediction'
image_sub_df[label_cols] = 0
image_sub_df

,id,PredictionString,file_name,2_class_prediction
1214,557a70442928_image,none 1 0 0 1 1,557a70442928.png,0
1215,36141cda67ad_image,none 1 0 0 1 1,36141cda67ad.png,0
1216,2413a23a5477_image,none 1 0 0 1 1,2413a23a5477.png,0
1217,c263b1e9aa64_image,none 1 0 0 1 1,c263b1e9aa64.png,0
1218,4fe0444d7fc5_image,none 1 0 0 1 1,4fe0444d7fc5.png,0
...,...,...,...,...
2472,46719b856de1_image,none 1 0 0 1 1,46719b856de1.png,0
2473,31c07523a69a_image,none 1 0 0 1 1,31c07523a69a.png,0
2474,f77d7d1aebab_image,none 1 0 0 1 1,f77d7d1aebab.png,0
2475,ccc5b63ca96d_image,none 1 0 0 1 1,ccc5b63ca96d.png,0


## EfficientNet for 2-class Predict

In [12]:
import albumentations
from albumentations.pytorch import ToTensorV2
from albumentations import RandomCrop,HorizontalFlip,VerticalFlip,Rotate,RandomBrightnessContrast,\
    RandomResizedCrop,Normalize,Resize,Compose,GaussianBlur,RandomBrightness,RandomContrast,RandomGamma,RandomRotate90,Transpose,\
    ShiftScaleRotate,Blur,OpticalDistortion,GridDistortion,HueSaturationValue,IAAAdditiveGaussianNoise,IAAPerspective,RandomSizedCrop,\
    RandomShadow,RandomSnow,RandomRain,RandomFog,CenterCrop,CoarseDropout,ChannelShuffle,ToGray,Cutout,PadIfNeeded,RandomCrop,VerticalFlip,HorizontalFlip,\
    Transpose,RandomRotate90,ShiftScaleRotate,ElasticTransform,GridDistortion,OpticalDistortion,RandomSizedCrop,HueSaturationValue,RGBShift,RandomBrightnessContrast,\
    RandomGamma,CLAHE,Blur,MedianBlur,MotionBlur,GaussNoise,GaussianBlur,RGBShift,RandomBrightnessContrast,IAAEmboss,IAASharpen,IAASuperpixels,RandomFog,RandomRain,\
    RandomShadow,RandomSnow,RandomSunFlare

transform_test = albumentations.Compose([
    Resize(args.image_size,args.image_size),
    Normalize()],
)

In [13]:
import cv2
from PIL import Image
from torchvision.transforms import ToTensor,Normalize,Compose,Resize
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,df,transform=None,mode:str='test'):
        self.df = df
        self.transform = transform
        self.mode = mode
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        img = Image.open(f'/kaggle/working/tmp/{split}/{row.file_name}').convert('RGB')
    
        img = self.transform(image=np.array(img))['image'].transpose(2,0,1)
        return img

In [14]:
from torch import adaptive_max_pool1d
from torch.utils.data import DataLoader
#effnetv2
import torch.nn as nn
import timm
from torch.nn import functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.nn import AvgPool2d,AdaptiveAvgPool2d


class Swish(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)
        
class Effnetv2(nn.Module):
    def __init__(self,pretrained=True):
        super().__init__()
        self.model = timm.create_model(args.arch_name, pretrained=pretrained ,num_classes=2, drop_rate = 0, drop_path_rate=0)
        self.logit = nn.Linear(self.model.classifier.in_features,2)
        self.preprocess = nn.Sequential(
            self.model.conv_stem,
            self.model.bn1,
            Swish()
        )
        self.blocks = self.model.blocks
        self.mask = nn.Sequential(
            nn.Conv2d(224, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )
        self.classifier = nn.Sequential(
             self.model.conv_head,
                self.model.bn2,
                Swish(),
        )

    def forward(self,x):
        batch_size = len(x)
        x = self.preprocess(x)
        for block in self.blocks[:5]:
            x = block(x)
        mask = self.mask(x)
        for block in self.blocks[5:]:
            x = block(x)
        x = self.classifier(x)
        x = AdaptiveAvgPool2d((1,1))(x)
        return self.logit(x.view(batch_size,-1)),mask

In [15]:
models_list = []

def load_study_model():
    for pth in args.model_image_path:
        model = Effnetv2(pretrained=False)
        print(pth)
        state = torch.load(pth, map_location='cuda:0') 
        model.load_state_dict(state)
        model.to(device);
        model.eval()
        models_list.append(model)
    print(len(models_list))

load_study_model()

/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold0_epoch9_ap0.8534853452608039.pth
/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold1_epoch6_ap0.8691063684343019.pth
/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold2_epoch4_ap0.8526443347470971.pth
/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold3_epoch9_ap0.8485065639200695.pth
/kaggle/input/efficientnet-model/efficientnet_model/image/best_fold4_epoch9_ap0.8745943231121824.pth
5


In [16]:
def test_func(models_list, test_loader):
    all_logits = []
    with torch.no_grad():
        for images in tqdm(test_loader): 
            images = images.to(device).float() 
            avg_model_preds = []
            for model in models_list:
                model.eval()
                logits1, mask1 = model(images)  
                logits2, mask2 = model(images.flip(-1))
                logits = (logits1.sigmoid().to('cpu').numpy() + logits2.sigmoid().to('cpu').numpy()) / 2
                avg_model_preds.append(logits)
            avg_model_preds = np.average(avg_model_preds, axis=0, weights=args.class_weight) ####
            all_logits.append(avg_model_preds)
        all_logits = np.concatenate(all_logits)
        return all_logits

In [17]:
test_dataset = MyDataset(image_sub_df,transform = transform_test,mode = 'test')
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers = args.n_worker, drop_last=False, pin_memory=True)

image_sub_df[label_cols] = test_func(models_list,test_dataloader)
image_sub_df

  0%|          | 0/106 [00:00<?, ?it/s]

,id,PredictionString,file_name,2_class_prediction
1214,557a70442928_image,none 1 0 0 1 1,557a70442928.png,0.620759
1215,36141cda67ad_image,none 1 0 0 1 1,36141cda67ad.png,0.456547
1216,2413a23a5477_image,none 1 0 0 1 1,2413a23a5477.png,0.026522
1217,c263b1e9aa64_image,none 1 0 0 1 1,c263b1e9aa64.png,0.076234
1218,4fe0444d7fc5_image,none 1 0 0 1 1,4fe0444d7fc5.png,0.131555
...,...,...,...,...
2472,46719b856de1_image,none 1 0 0 1 1,46719b856de1.png,0.711086
2473,31c07523a69a_image,none 1 0 0 1 1,31c07523a69a.png,0.030373
2474,f77d7d1aebab_image,none 1 0 0 1 1,f77d7d1aebab.png,0.845663
2475,ccc5b63ca96d_image,none 1 0 0 1 1,ccc5b63ca96d.png,0.995428


In [18]:
image_sub_df = image_sub_df.merge(meta,how="left",on='id')
image_sub_df["none"] = 1-image_sub_df[label_cols]
image_sub_df

,id,PredictionString,file_name,2_class_prediction,dim0,dim1,split,study_id,none
0,557a70442928_image,none 1 0 0 1 1,557a70442928.png,0.620759,3320,3408,test,795051254905_study,0.379241
1,36141cda67ad_image,none 1 0 0 1 1,36141cda67ad.png,0.456547,3480,4240,test,0d476d070d71_study,0.543453
2,2413a23a5477_image,none 1 0 0 1 1,2413a23a5477.png,0.026522,2320,2832,test,4996bf5117c7_study,0.973478
3,c263b1e9aa64_image,none 1 0 0 1 1,c263b1e9aa64.png,0.076234,2801,2802,test,8b273337a684_study,0.923766
4,4fe0444d7fc5_image,none 1 0 0 1 1,4fe0444d7fc5.png,0.131555,2392,3014,test,aafc8126d5a1_study,0.868445
...,...,...,...,...,...,...,...,...,...
1258,46719b856de1_image,none 1 0 0 1 1,46719b856de1.png,0.711086,2648,2867,test,ee860264dd8c_study,0.288914
1259,31c07523a69a_image,none 1 0 0 1 1,31c07523a69a.png,0.030373,3488,4256,test,81c860c6efe8_study,0.969627
1260,f77d7d1aebab_image,none 1 0 0 1 1,f77d7d1aebab.png,0.845663,3052,3012,test,e6e02ec8aff5_study,0.154337
1261,ccc5b63ca96d_image,none 1 0 0 1 1,ccc5b63ca96d.png,0.995428,2520,3032,test,b93bfa119338_study,0.004572


In [19]:
del models_list
torch.cuda.empty_cache()
gc.collect()

64

## Yolo Predict

In [20]:
yolo_df = all_df[study_len:].reset_index(drop=True)
yolo_df = pd.merge(yolo_df, meta, on='id', how='left')
yolo_df = yolo_df.drop(['PredictionString'], axis=1)
yolo_df

,id,id_last_str,dim0,dim1,split,study_id
0,557a70442928_image,e,3320,3408,test,795051254905_study
1,36141cda67ad_image,e,3480,4240,test,0d476d070d71_study
2,2413a23a5477_image,e,2320,2832,test,4996bf5117c7_study
3,c263b1e9aa64_image,e,2801,2802,test,8b273337a684_study
4,4fe0444d7fc5_image,e,2392,3014,test,aafc8126d5a1_study
...,...,...,...,...,...,...
1258,46719b856de1_image,e,2648,2867,test,ee860264dd8c_study
1259,31c07523a69a_image,e,3488,4256,test,81c860c6efe8_study
1260,f77d7d1aebab_image,e,3052,3012,test,e6e02ec8aff5_study
1261,ccc5b63ca96d_image,e,2520,3032,test,b93bfa119338_study


In [21]:
from ultralytics import YOLO
from PIL import Image
import cv2

for fold in range(5):
    model_path = f'/kaggle/input/yolov8/folds/fold{str(fold)}_best.pt'
    model = YOLO(model_path)
    model.predict(source=f'/kaggle/working/tmp/{split}',name=f'fold{str(fold)}',imgsz=512, augment=True, conf=0.001, iou=0.5, save_txt=True, save_conf=True)

model_path = '/kaggle/input/yolov8/yolov8.pt'
model = YOLO(model_path)
result = model.predict(source=f'/kaggle/working/tmp/{split}',name='best',imgsz=512, augment=True, conf=0.001, iou=0.5, save_txt=True, save_conf=True)



    WARNING ⚠️ stream/video/webcam/dir predict source will accumulate results in RAM unless `stream=True` is passed,
    causing potential out-of-memory errors for large sources or long-running streams/videos.

    Usage:
        results = model(source=..., stream=True)  # generator of Results objects
        for r in results:
            boxes = r.boxes  # Boxes object for bbox outputs
            masks = r.masks  # Masks object for segment masks outputs
            probs = r.probs  # Class probabilities for classification outputs

image 1/1263 /kaggle/working/tmp/test/0026720152f5.png: 512x512 32 opacitys, 153.5ms
image 2/1263 /kaggle/working/tmp/test/00fc8fc35dc1.png: 512x512 12 opacitys, 73.3ms
image 3/1263 /kaggle/working/tmp/test/01c3512eebc3.png: 512x512 22 opacitys, 71.3ms
image 4/1263 /kaggle/working/tmp/test/01f948f8e544.png: 512x512 22 opacitys, 67.9ms
image 5/1263 /kaggle/working/tmp/test/022146012034.png: 512x512 11 opacitys, 67.1ms
image 6/1263 /kaggle/working/tmp/test/

In [22]:
def yolo2voc(bboxes):
    bboxes = bboxes.copy().astype(float)
    bboxes[..., [0, 1]] = bboxes[..., [0, 1]] - bboxes[..., [2, 3]]/2
    bboxes[..., [2, 3]] = bboxes[..., [0, 1]] + bboxes[..., [2, 3]]
    return bboxes

def yolo2voc1(image_height, image_width, bboxes):
    bboxes = bboxes.copy().astype(float)
    bboxes[..., [0, 2]] = bboxes[..., [0, 2]]* image_width
    bboxes[..., [1, 3]] = bboxes[..., [1, 3]]* image_height
    return bboxes

In [23]:
from ensemble_boxes import weighted_boxes_fusion
image_ids = []
PredictionStrings = []

for i in range(len(yolo_df)):
    is_exist1 = True
    is_exist2 = True
    
    iou_thr = 0.5
    skip_box_thr = 0.0001
    image_id = yolo_df.loc[i,'id']
    w, h = yolo_df.loc[yolo_df.id==image_id,['dim1', 'dim0']].values[0]

    weights = [1, 1, 1, 1, 1]

    boxes_list = []
    scores_list = []
    labels_list = []

    yolo_save_dirs = [f'/kaggle/working/runs/detect/fold{fold}/labels' for fold in range(5)]
    
    for save_dir in yolo_save_dirs:
        pth = save_dir + '/{}.txt'
        file_path = pth.format(image_id.replace('_image', ''))
        if os.path.exists(file_path):
            f = open(file_path, 'r')
            data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
            data = data[:, [0, 5, 1, 2, 3, 4]]
            data[:,2:] = yolo2voc(data[:, 2:])
            boxes_list.append(data[:,2:].tolist())
            scores_list.append(data[:,1].tolist())
            labels_list.append(data[:,0].tolist())

    if len(boxes_list)>1:
        if len(boxes_list)==2:
            weights = [1, 1]
        elif len(boxes_list)==3:
            weights = [1, 1, 1]
        elif len(boxes_list)==4:
            weights = [1, 1, 1, 1]
        else:
            weights = [1, 1, 2, 1, 1]
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        data1 = np.zeros((boxes.shape[0],6))
        data1[:,0] = labels
        data1[:,1] = scores
        data1[:,2:] = boxes
    elif len(boxes_list)==1:
        data1 = data
    else:
        is_exist1 = False


    boxes_list = []
    scores_list = []
    labels_list = []

    file_path = '/kaggle/working/runs/detect/best/labels/{}.txt'.format(image_id.replace('_image', ''))
    if os.path.exists(file_path):
        f = open(file_path, 'r')
        data = np.array(f.read().replace('\n', ' ').strip().split(' ')).astype(np.float32).reshape(-1, 6)
        data = data[:, [0, 5, 1, 2, 3, 4]]
        data[:,2:] = yolo2voc(data[:, 2:])

    weights = [1, 6]
    if os.path.exists(file_path) and is_exist1:
        boxes_list.append(data[:,2:].tolist())
        scores_list.append(data[:,1].tolist())
        labels_list.append(data[:,0].tolist())
        boxes_list.append(data1[:,2:].tolist())
        scores_list.append(data1[:,1].tolist())
        labels_list.append(data1[:,0].tolist())
        boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
        data2 = np.zeros((boxes.shape[0],6))
        data2[:,0] = labels
        data2[:,1] = scores
        data2[:,2:] = boxes
    elif os.path.exists(file_path) or is_exist1:
        data2 = data
    else:
        is_exist2 = False
    if is_exist1==False and is_exist2==False:
        continue
            
    bboxes = list(np.round(np.concatenate((data2[:, :2], np.round(yolo2voc1(h, w, data2[:, 2:]))), axis =1).reshape(-1), 12).astype(str))
    for idx in range(len(bboxes)):
        bboxes[idx] = str(int(float(bboxes[idx]))) if idx%6!=1 else bboxes[idx]
    image_ids.append(image_id)
    PredictionStrings.append(' '.join(bboxes))

yolo_predict_df = pd.DataFrame({'id':image_ids, 'PredictionString':PredictionStrings})
yolo_predict_df

,id,PredictionString
0,557a70442928_image,1 0.13438635213 2303 1793 2960 2211 1 0.016745...
1,36141cda67ad_image,1 0.095072984695 3145 1447 3719 2422 1 0.04068...
2,2413a23a5477_image,1 0.298962082182 1657 780 2421 1718 1 0.202971...
3,c263b1e9aa64_image,1 0.288801363536 408 989 1119 1918 1 0.2477065...
4,4fe0444d7fc5_image,1 0.177460977009 1989 870 2865 1598 1 0.115512...
...,...,...
1258,46719b856de1_image,1 0.028545443501 1665 1003 2406 1829 1 0.02500...
1259,31c07523a69a_image,1 0.530465875353 2769 732 4074 2811 1 0.314306...
1260,f77d7d1aebab_image,1 0.015601596662 1867 1538 2432 2092 1 0.01106...
1261,ccc5b63ca96d_image,1 0.002934679921 1937 711 2944 2313 1 0.001166...


In [24]:
yolo_df = pd.merge(yolo_df, yolo_predict_df, on = 'id', how = 'left').fillna("none 1 0 0 1 1")
yolo_df = yolo_df[['id', 'PredictionString']]
yolo_df

,id,PredictionString
0,557a70442928_image,1 0.13438635213 2303 1793 2960 2211 1 0.016745...
1,36141cda67ad_image,1 0.095072984695 3145 1447 3719 2422 1 0.04068...
2,2413a23a5477_image,1 0.298962082182 1657 780 2421 1718 1 0.202971...
3,c263b1e9aa64_image,1 0.288801363536 408 989 1119 1918 1 0.2477065...
4,4fe0444d7fc5_image,1 0.177460977009 1989 870 2865 1598 1 0.115512...
...,...,...
1258,46719b856de1_image,1 0.028545443501 1665 1003 2406 1829 1 0.02500...
1259,31c07523a69a_image,1 0.530465875353 2769 732 4074 2811 1 0.314306...
1260,f77d7d1aebab_image,1 0.015601596662 1867 1538 2432 2092 1 0.01106...
1261,ccc5b63ca96d_image,1 0.002934679921 1937 711 2944 2313 1 0.001166...


In [25]:
for i in range(yolo_df.shape[0]):
    if yolo_df.loc[i,'PredictionString'] == "none 1 0 0 1 1":
        continue
    sub_df_split = yolo_df.loc[i,'PredictionString'].split()
    sub_df_list = []
    for j in range(int(len(sub_df_split) / 6)):
        sub_df_list.append('opacity')
        sub_df_list.append(sub_df_split[6 * j + 1])
        sub_df_list.append(sub_df_split[6 * j + 2])
        sub_df_list.append(sub_df_split[6 * j + 3])
        sub_df_list.append(sub_df_split[6 * j + 4])
        sub_df_list.append(sub_df_split[6 * j + 5])
    yolo_df.loc[i,'PredictionString'] = ' '.join(sub_df_list)

yolo_df['none'] = image_sub_df['none'] 
for i in range(yolo_df.shape[0]):
    if yolo_df.loc[i,'PredictionString'] != 'none 1 0 0 1 1':
        yolo_df.loc[i,'PredictionString'] = yolo_df.loc[i,'PredictionString'] + ' none ' + str(yolo_df.loc[i,'none']) + ' 0 0 1 1'

yolo_df = yolo_df[['id', 'PredictionString']] 
# yolo_df.to_csv('yolo_df.csv', index=False)
image_df = yolo_df
image_df

,id,PredictionString
0,557a70442928_image,opacity 0.13438635213 2303 1793 2960 2211 opac...
1,36141cda67ad_image,opacity 0.095072984695 3145 1447 3719 2422 opa...
2,2413a23a5477_image,opacity 0.298962082182 1657 780 2421 1718 opac...
3,c263b1e9aa64_image,opacity 0.288801363536 408 989 1119 1918 opaci...
4,4fe0444d7fc5_image,opacity 0.177460977009 1989 870 2865 1598 opac...
...,...,...
1258,46719b856de1_image,opacity 0.028545443501 1665 1003 2406 1829 opa...
1259,31c07523a69a_image,opacity 0.530465875353 2769 732 4074 2811 opac...
1260,f77d7d1aebab_image,opacity 0.015601596662 1867 1538 2432 2092 opa...
1261,ccc5b63ca96d_image,opacity 0.002934679921 1937 711 2944 2313 opac...


# 3. 4-class Predict on Study Level

In [26]:
study_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
study_df = study_df[:study_len]
study_df

,id,PredictionString
0,00188a671292_study,negative 1 0 0 1 1
1,004bd59708be_study,negative 1 0 0 1 1
2,00508faccd39_study,negative 1 0 0 1 1
3,006486aa80b2_study,negative 1 0 0 1 1
4,00655178fdfc_study,negative 1 0 0 1 1
...,...,...
1209,ff1ba0e9aaf0_study,negative 1 0 0 1 1
1210,ff2cc4de58c5_study,negative 1 0 0 1 1
1211,ff2f0a744930_study,negative 1 0 0 1 1
1212,ff88940dce8b_study,negative 1 0 0 1 1


In [27]:
meta_study = meta.drop_duplicates(['study_id'], keep='last')
meta_study.columns = ['image_id', 'dim0', 'dim1', 'split', 'id']
meta_study

,image_id,dim0,dim1,split,id
0,a29c5a68b07b_image,2320,2828,test,2fb11712bc93_study
1,9850b5470fd6_image,2330,2382,test,19c66935e737_study
2,8d6dea06a032_image,2422,3344,test,2fc50bf199cd_study
3,dfc5c09a50bc_image,1140,1387,test,97c5d6eb413d_study
4,7230234e120a_image,2318,2383,test,593c3f815635_study
...,...,...,...,...,...
1257,cede944b70a5_image,2539,3050,test,57e2c6e368f2_study
1258,a43200bd5ceb_image,2336,2836,test,b12d2e5c39b7_study
1260,b30d2aef985f_image,3480,4240,test,6484393291ea_study
1261,a37a362df0ac_image,3480,4240,test,531aa20ff7c3_study


In [28]:
study_df = pd.read_csv('/kaggle/input/siim-covid19-detection/sample_submission.csv')
study_df = study_df[:study_len]

study_df = pd.merge(study_df, meta_study, on='id', how='left')

study_df['file_name'] = study_df['image_id'].str.replace('_image', '') + ".png"
study_df = study_df[['id', 'PredictionString', 'file_name']]

study_df['negative'] = 0
study_df['typical'] = 0
study_df['indeterminate'] = 0
study_df['atypical'] = 0

label_cols = study_df.columns[3:]

study_df

,id,PredictionString,file_name,negative,typical,indeterminate,atypical
0,00188a671292_study,negative 1 0 0 1 1,3dcdfc352a06.png,0,0,0,0
1,004bd59708be_study,negative 1 0 0 1 1,c39146cbda47.png,0,0,0,0
2,00508faccd39_study,negative 1 0 0 1 1,951211f8e1bb.png,0,0,0,0
3,006486aa80b2_study,negative 1 0 0 1 1,5e0e7acd9c7d.png,0,0,0,0
4,00655178fdfc_study,negative 1 0 0 1 1,5b8ee5baa1d5.png,0,0,0,0
...,...,...,...,...,...,...,...
1209,ff1ba0e9aaf0_study,negative 1 0 0 1 1,064b37b01cd2.png,0,0,0,0
1210,ff2cc4de58c5_study,negative 1 0 0 1 1,02eceb0fc405.png,0,0,0,0
1211,ff2f0a744930_study,negative 1 0 0 1 1,16a0d37747d7.png,0,0,0,0
1212,ff88940dce8b_study,negative 1 0 0 1 1,aa55fc6c616c.png,0,0,0,0


In [29]:
import albumentations
from albumentations.pytorch import ToTensorV2
from albumentations import RandomCrop,HorizontalFlip,VerticalFlip,Rotate,RandomBrightnessContrast,\
    RandomResizedCrop,Normalize,Resize,Compose,GaussianBlur,RandomBrightness,RandomContrast,RandomGamma,RandomRotate90,Transpose,\
    ShiftScaleRotate,Blur,OpticalDistortion,GridDistortion,HueSaturationValue,IAAAdditiveGaussianNoise,IAAPerspective,RandomSizedCrop,\
    RandomShadow,RandomSnow,RandomRain,RandomFog,CenterCrop,CoarseDropout,ChannelShuffle,ToGray,Cutout,PadIfNeeded,RandomCrop,VerticalFlip,HorizontalFlip,\
    Transpose,RandomRotate90,ShiftScaleRotate,ElasticTransform,GridDistortion,OpticalDistortion,RandomSizedCrop,HueSaturationValue,RGBShift,RandomBrightnessContrast,\
    RandomGamma,CLAHE,Blur,MedianBlur,MotionBlur,GaussNoise,GaussianBlur,RGBShift,RandomBrightnessContrast,IAAEmboss,IAASharpen,IAASuperpixels,RandomFog,RandomRain,\
    RandomShadow,RandomSnow,RandomSunFlare

transform_test = albumentations.Compose([
    Resize(args.image_size,args.image_size),
    Normalize()],
)

In [30]:
import cv2
from PIL import Image
from torchvision.transforms import ToTensor,Normalize,Compose,Resize
class MyDataset(torch.utils.data.Dataset):
    def __init__(self,df,transform=None,mode:str='test'):
        self.df = df
        self.transform = transform
        self.mode = mode
    def __len__(self):
        return len(self.df)
    def __getitem__(self,idx):
        row = self.df.iloc[idx]
        img = Image.open(f'/kaggle/working/tmp/{split}/{row.file_name}').convert('RGB')
    
        img = self.transform(image=np.array(img))['image'].transpose(2,0,1)
        return img

In [31]:
from torch import adaptive_max_pool1d
from torch.utils.data import DataLoader
#effnetv2
import torch.nn as nn
import timm
from torch.nn import functional as F
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingLR
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
from torch.nn import AvgPool2d,AdaptiveAvgPool2d


class Swish(nn.Module):
        def forward(self, x):
            return x * torch.sigmoid(x)
        
class Effnetv2(nn.Module):
    def __init__(self,pretrained=True):
        super().__init__()
        self.model = timm.create_model(args.arch_name,pretrained=pretrained,num_classes=4,drop_rate = 0,drop_path_rate=0)
        self.logit = nn.Linear(self.model.classifier.in_features,4)
        self.preprocess = nn.Sequential(
            self.model.conv_stem,
            self.model.bn1,
            Swish()
        )
        self.blocks = self.model.blocks
        self.mask = nn.Sequential(
            nn.Conv2d(224, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 1, kernel_size=1, padding=0),
        )
        self.classifier = nn.Sequential(
             self.model.conv_head,
                self.model.bn2,
                Swish(),
        )

    def forward(self,x):
        batch_size = len(x)
        x = self.preprocess(x)
        for block in self.blocks[:5]:
            x = block(x)
        mask = self.mask(x)
        for block in self.blocks[5:]:
            x = block(x)
        x = self.classifier(x)
        x = AdaptiveAvgPool2d((1,1))(x)
        return self.logit(x.view(batch_size,-1)),mask

In [32]:
models_list = []

def load_study_model():
    for pth in args.model_study_path:
        model = Effnetv2(pretrained=False)
        print(pth)
        state = torch.load(pth, map_location='cuda:0') 
        model.load_state_dict(state)
        model.to(device);
        model.eval()
        models_list.append(model)
    print(len(models_list))
    
load_study_model()

/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold0_epoch5_ap0.5511132228812636.pth
/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold1_epoch9_ap0.5633224707416896.pth
/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold2_epoch9_ap0.5755307073313223.pth
/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold3_epoch10_ap0.565790786617794.pth
/kaggle/input/efficientnet-model/efficientnet_model/study/best_fold4_epoch4_ap0.5755384751101597.pth
5


In [33]:
def test_func(models_list, test_loader):
    all_logits = []
    with torch.no_grad():
        for images in tqdm(test_loader): 
            images = images.to(device).float() 
            avg_model_preds = []
            for model in models_list:
                model.eval()
                logits1, mask1 = model(images)  
                logits2, mask2 = model(images.flip(-1))
                logits = (logits1.sigmoid().to('cpu').numpy() + logits2.sigmoid().to('cpu').numpy()) / 2
                avg_model_preds.append(logits)
            avg_model_preds = np.average(avg_model_preds, axis=0, weights=args.class_weight) ####
            all_logits.append(avg_model_preds)
        all_logits = np.concatenate(all_logits)
        return all_logits

In [34]:
test_dataset = MyDataset(study_df,transform = transform_test,mode = 'test')

test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=args.batch_size, shuffle=False, num_workers = args.n_worker, drop_last=False, pin_memory=True)

study_df[label_cols] = test_func(models_list,test_dataloader)
study_df = study_df.drop("file_name",axis=1)
study_df.columns = ['id', 'PredictionString1', 'negative', 'typical', 'indeterminate', 'atypical']
all_df = pd.merge(all_df, study_df, on = 'id', how = 'left')
all_df

  0%|          | 0/102 [00:00<?, ?it/s]

,id,PredictionString,id_last_str,PredictionString1,negative,typical,indeterminate,atypical
0,00188a671292_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.830572,0.057769,0.100136,0.029516
1,004bd59708be_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.002714,0.871412,0.079355,0.021157
2,00508faccd39_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.646659,0.089425,0.162758,0.107884
3,006486aa80b2_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.423098,0.201437,0.299594,0.079184
4,00655178fdfc_study,negative 1 0 0 1 1,y,negative 1 0 0 1 1,0.438876,0.240200,0.251870,0.057067
...,...,...,...,...,...,...,...,...
2472,46719b856de1_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN
2473,31c07523a69a_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN
2474,f77d7d1aebab_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN
2475,ccc5b63ca96d_image,none 1 0 0 1 1,e,NaN,NaN,NaN,NaN,NaN


In [35]:
for i in range(study_len):
    negative = all_df.loc[i,'negative']
    typical = all_df.loc[i,'typical']
    indeterminate = all_df.loc[i,'indeterminate']
    atypical = all_df.loc[i,'atypical']
    all_df.loc[i, 'PredictionString'] = f'negative {negative} 0 0 1 1 typical {typical} 0 0 1 1 indeterminate {indeterminate} 0 0 1 1 atypical {atypical} 0 0 1 1'
    
study_df = all_df[['id', 'PredictionString']][:study_len]
# study_df.to_csv('study_df.csv', index=False)
study_df

,id,PredictionString
0,00188a671292_study,negative 0.8305720925331116 0 0 1 1 typical 0....
1,004bd59708be_study,negative 0.002713597250476596 0 0 1 1 typical ...
2,00508faccd39_study,negative 0.6466592609882355 0 0 1 1 typical 0....
3,006486aa80b2_study,negative 0.4230982720851898 0 0 1 1 typical 0....
4,00655178fdfc_study,negative 0.4388757586479187 0 0 1 1 typical 0....
...,...,...
1209,ff1ba0e9aaf0_study,negative 0.2660233244299889 0 0 1 1 typical 0....
1210,ff2cc4de58c5_study,negative 0.09476278200745583 0 0 1 1 typical 0...
1211,ff2f0a744930_study,negative 0.0008154470968293026 0 0 1 1 typical...
1212,ff88940dce8b_study,negative 0.619315892457962 0 0 1 1 typical 0.1...


In [36]:
del models_list
torch.cuda.empty_cache()
gc.collect()

43

# 4. Merge Study and Image

In [37]:
image_df = meta.merge(yolo_df,on="id")
image_df

,id,dim0,dim1,split,study_id,PredictionString
0,a29c5a68b07b_image,2320,2828,test,2fb11712bc93_study,opacity 0.453987121582 653 531 1488 1907 opaci...
1,9850b5470fd6_image,2330,2382,test,19c66935e737_study,opacity 0.30024102756 263 680 1017 1906 opacit...
2,8d6dea06a032_image,2422,3344,test,2fc50bf199cd_study,opacity 0.119990297726 296 1191 962 2197 opaci...
3,dfc5c09a50bc_image,1140,1387,test,97c5d6eb413d_study,opacity 0.064530415194 879 446 1141 759 opacit...
4,7230234e120a_image,2318,2383,test,593c3f815635_study,opacity 0.48258297784 1443 1153 2147 1887 opac...
...,...,...,...,...,...,...
1258,a43200bd5ceb_image,2336,2836,test,b12d2e5c39b7_study,opacity 0.225861719676 1528 315 2221 1676 opac...
1259,37bf83df1b86_image,3480,4240,test,6484393291ea_study,opacity 0.266430905887 2375 453 3500 2069 opac...
1260,b30d2aef985f_image,3480,4240,test,6484393291ea_study,opacity 0.365218060357 739 473 1897 2090 opaci...
1261,a37a362df0ac_image,3480,4240,test,531aa20ff7c3_study,opacity 0.034480933632 612 1145 1769 2177 opac...


In [38]:
from ensemble_boxes import *

weights = [3, 1]
iou_thr = 0.6
skip_box_thr = 0.0001
image_output_list = []

for index, id_, dim0, dim1, split, study_id, yolo5_boxes in image_df.itertuples():
    yolo5_boxes_list = yolo5_boxes.split(" ")
    assert len(yolo5_boxes_list) % 6 == 0
    none_list = yolo5_boxes_list[-6:]
    yolo5_boxes_list = yolo5_boxes_list[:-6]
    boxes_list = [[],[]]
    scores_list = [[],[]]
    for i in range(len(yolo5_boxes_list)):
        if i % 6 == 1:
            scores_list[0].append(float(yolo5_boxes_list[i]))
        if i % 6 == 2:
            boxes_list[0].append([float(yolo5_boxes_list[i])/dim1,
                                  float(yolo5_boxes_list[i+1])/dim0,
                                  float(yolo5_boxes_list[i+2])/dim1,
                                  float(yolo5_boxes_list[i+3])/dim0])

            
    labels_list = [[0]*len(scores_list[0]), [0]*len(scores_list[1])]        
    boxes, scores, labels = weighted_boxes_fusion(boxes_list, scores_list, labels_list, weights=weights, iou_thr=iou_thr, skip_box_thr=skip_box_thr)
    
    output_str = ""
    for box, score in zip(boxes, scores): 
        output_str += "opacity "
        output_str += f"{score} "
        for b_i in range(4):
            if b_i in [0,2]:
                output_str += f"{int(round(box[b_i]*dim1,0))} "
            elif b_i in [1,3]:
                output_str += f"{int(round(box[b_i]*dim0,0))} "

    output_str += " ".join(none_list)
    image_output_list.append(output_str)

    
image_df["PredictionString"] = image_output_list
image_df = image_df[["id", "PredictionString"]]
image_df

,id,PredictionString
0,a29c5a68b07b_image,opacity 0.3404903411865 653 531 1488 1907 opac...
1,9850b5470fd6_image,opacity 0.22518077066999997 263 680 1017 1906 ...
2,8d6dea06a032_image,opacity 0.0899927232945 296 1191 962 2197 opac...
3,dfc5c09a50bc_image,opacity 0.04839781139550001 879 446 1141 759 o...
4,7230234e120a_image,opacity 0.36193723338 1443 1153 2147 1887 opac...
...,...,...
1258,a43200bd5ceb_image,opacity 0.169396289757 1528 315 2221 1676 opac...
1259,37bf83df1b86_image,opacity 0.19982317941525002 2375 453 3500 2069...
1260,b30d2aef985f_image,opacity 0.27391354526775 739 473 1897 2090 opa...
1261,a37a362df0ac_image,opacity 0.025860700224 612 1145 1769 2177 opac...


In [39]:
submission_df = study_df.append(image_df).reset_index(drop=True)
submission_df.to_csv('/kaggle/working/submission.csv',index = False)
submission_df

,id,PredictionString
0,00188a671292_study,negative 0.8305720925331116 0 0 1 1 typical 0....
1,004bd59708be_study,negative 0.002713597250476596 0 0 1 1 typical ...
2,00508faccd39_study,negative 0.6466592609882355 0 0 1 1 typical 0....
3,006486aa80b2_study,negative 0.4230982720851898 0 0 1 1 typical 0....
4,00655178fdfc_study,negative 0.4388757586479187 0 0 1 1 typical 0....
...,...,...
2472,a43200bd5ceb_image,opacity 0.169396289757 1528 315 2221 1676 opac...
2473,37bf83df1b86_image,opacity 0.19982317941525002 2375 453 3500 2069...
2474,b30d2aef985f_image,opacity 0.27391354526775 739 473 1897 2090 opa...
2475,a37a362df0ac_image,opacity 0.025860700224 612 1145 1769 2177 opac...


In [40]:
shutil.rmtree('/kaggle/working/runs')